# Default strategies insertion into MongoDB

In [8]:
## Libraries
import os
import pkgutil
from importlib import import_module
import inspect
from backtesting import Strategy

from pymongo import MongoClient

### Database connection

In [23]:
## DATABASE
# Establish a connection to MongoDB
client = MongoClient('mongodb://localhost:27017/')

# Access the collection for the strategies
db = client['strategies']

In [24]:
# Delete the existing items in the collection to start fresh
db['default_strategies'].delete_many({})

### Strategy classes definition

In [25]:
# Get the directory where the strategy modules are located
modules_dir = 'modules'

# Create an empty dictionary to store the strategy classes
strategy_classes = {}

# Iterate over the strategy modules in the directory
for _, module_name, _ in pkgutil.iter_modules([modules_dir]):
    # Import the strategy module dynamically
    module = import_module(f'{modules_dir}.{module_name}')

    # Get the strategy class from the module
    strategy_class = next((cls for _, cls in inspect.getmembers(module, inspect.isclass) if issubclass(cls, Strategy)), None)

    # Add the strategy class to the dictionary with the module name as key
    if strategy_class is not None:
        strategy_classes[module_name] = strategy_class

In [26]:
strategy_classes

{'AverageDirectionalMovement': modules.AverageDirectionalMovement.AverageDirectionalMovement,
 'BollingerBands': modules.BollingerBands.BollingerBands,
 'Breakout': modules.Breakout.Breakout,
 'MACD': modules.MACD.MACD,
 'MeanReversion': modules.MeanReversion.MeanReversion,
 'MeanReversionBollinger': modules.MeanReversionBollinger.MeanReversionBollinger,
 'MomentumVolatility': modules.MomentumVolatility.MomentumVolatility,
 'MovingAverageCrossover': modules.MovingAverageCrossover.MovingAverageCrossover,
 'RelativeStrengthIndex': modules.RelativeStrengthIndex.RelativeStrengthIndex,
 'Turtle': backtesting.backtesting.Strategy,
 'VolatilityBreakout': backtesting.backtesting.Strategy}

### Description of the strategies

In [27]:
# Create a dictionary with the descriptions of each strategy
strategy_descriptions = {
    "MeanReversion": "Mean Reversion strategy aims to capture price movements that deviate from the mean. It buys when the price is below the mean and sells when the price is above the mean.",
    "MomentumVolatility": "Momentum strategy aims to capture trends in the market. It buys when the price shows positive momentum and sells when the price shows negative momentum.",
    "MovingAverageCrossover": "Moving Average Crossover strategy uses two moving averages of different periods. It buys when the shorter-term moving average crosses above the longer-term moving average, and sells when the shorter-term moving average crosses below the longer-term moving average.",
    "Breakout": "Breakout strategy aims to capture price breakouts from a defined range. It buys when the price breaks above the range and sells when the price breaks below the range.",
    "RelativeStrengthIndex": "RSI strategy uses the Relative Strength Index (RSI) indicator to identify overbought and oversold conditions. It buys when the RSI crosses below a certain threshold and sells when the RSI crosses above another threshold.",
    "BollingerBands": "Bollinger Bands strategy uses the Bollinger Bands indicator to identify potential buying and selling opportunities. It buys when the price touches the lower Bollinger Band and sells when the price touches the upper Bollinger Band.",
    "MeanReversionBollinger": "Mean Reversion with Bollinger Bands strategy combines mean reversion and Bollinger Bands. It buys when the price deviates below the mean and touches the lower Bollinger Band, and sells when the price deviates above the mean and touches the upper Bollinger Band.",
    "MACD": "MACD strategy uses the Moving Average Convergence Divergence (MACD) indicator to generate buy and sell signals based on the crossovers of the MACD line and the signal line.",
    "AverageDirectionalMovement": "ADX strategy uses the Average Directional Index (ADX) indicator to determine the strength of a trend. It buys when the ADX is above a certain threshold, indicating a strong trend, and sells when the ADX is below another threshold.",
    "Turtle": "Turtle Trading strategy involves entering long positions when the price exceeds the highest high over a certain lookback period and entering short positions when the price falls below the lowest low over the same period.",
    "VolatilityBreakout": "Volatility Breakout strategy aims to capture price breakouts during periods of high volatility. It buys when the price exceeds the highest high multiplied by a volatility factor and sells when the price falls below the lowest low multiplied by the same factor."
}

### Description of the tunable parameters

In [28]:
# Create a dictionary with the parameters and descriptions of each strategy
strategy_parameters = {
    "MeanReversion": {
        'lookback_period': {
            'description': 'The number of periods to look back for calculating mean reversion. It measures how far the current price deviates from its mean value over the specified period. If the price deviates too much, it triggers buying or selling.'
        },
        'z_score_threshold': {
            'description': 'The threshold value for the z-score. The z-score measures the number of standard deviations the current price is away from its mean value. If the z-score exceeds this threshold, it triggers buying or selling.'
        }
    },
    "MomentumVolatility": {
        'lookback_period': {
            'description': 'The number of periods to look back for calculating momentum. It measures the percentage change in the price over the specified period. If the recent price change is positive, it triggers buying. If the recent price change is negative, it triggers selling.'
        }
    },
    "MovingAverageCrossover": {
        'short_period': {
            'description': 'The number of periods for the short-term moving average. It calculates the average price over the specified period. If the short-term moving average crosses above the long-term moving average, it triggers buying. If it crosses below, it triggers selling.'
        },
        'long_period': {
            'description': 'The number of periods for the long-term moving average. It calculates the average price over the specified period. If the short-term moving average crosses above the long-term moving average, it triggers buying. If it crosses below, it triggers selling.'
        }
    },
    "Breakout": {
        'lookback_period': {
            'description': 'The number of periods to look back for calculating the range breakout. It identifies the highest high and lowest low over the specified period. If the current price breaks above the range, it triggers buying. If it breaks below, it triggers selling.'
        }
    },
    "RelativeStrengthIndex": {
        'rsi_period': {
            'description': 'The number of periods for calculating the Relative Strength Index (RSI). It measures the magnitude of recent price changes to evaluate overbought and oversold conditions. If the RSI exceeds the sell threshold, it triggers selling. If it falls below the buy threshold, it triggers buying.'
        },
        'sell_threshold': {
            'description': 'The threshold value for the RSI to trigger selling. If the RSI exceeds this value, it indicates an overbought condition and triggers selling.'
        },
        'buy_threshold': {
            'description': 'The threshold value for the RSI to trigger buying. If the RSI falls below this value, it indicates an oversold condition and triggers buying.'
        }
    },
    "BollingerBands": {
        'period': {
            'description': 'The number of periods for calculating the Bollinger Bands. They consist of a moving average and an upper and lower band that represent the price volatility. If the price falls below the lower band, it triggers buying. If it exceeds the upper band, it triggers selling.'
        },
        'std_devs': {
            'description': 'The number of standard deviations for the Bollinger Bands. It determines the width of the bands and reflects the price volatility. Higher standard deviations result in wider bands.'
        }
    },
    "MeanReversionBollinger": {
        'lookback_period': {
            'description': 'The number of periods to look back for calculating mean reversion. It measures how far the current price deviates from its mean value over the specified period. If the price deviates too much and touches the upper or lower Bollinger Band, it triggers buying or selling.'
        },
        'z_score_threshold': {
            'description': 'The threshold value for the z-score. The z-score measures the number of standard deviations the current price is away from its mean value. If the z-score exceeds this threshold and the price touches the upper or lower Bollinger Band, it triggers buying or selling.'
        }
    },
    "MACD": {
        'fast_period': {
            'description': 'The number of periods for the fast Exponential Moving Average (EMA) in MACD calculation. It determines the sensitivity of the fast EMA to recent price changes.'
        },
        'slow_period': {
            'description': 'The number of periods for the slow Exponential Moving Average (EMA) in MACD calculation. It determines the sensitivity of the slow EMA to recent price changes.'
        },
        'signal_period': {
            'description': 'The number of periods for the signal line in MACD calculation. It represents the smoothing of the MACD line and helps generate buy and sell signals.'
        }
    },
    "AverageDirectionalMovement": {
        'adx_period': {
            'description': 'The number of periods for calculating the Average Directional Movement Index (ADX). It measures the strength of a trend. If the ADX exceeds the threshold, it indicates a strong trend and triggers buying.'
        },
        'adx_threshold': {
            'description': 'The threshold value for the ADX to trigger buying. If the ADX exceeds this value, it indicates a strong trend and triggers buying.'
        }
    },
    "Turtle": {
        'lookback_period': {
            'description': 'The number of periods to look back for calculating the highest high and lowest low. It identifies the highest high and lowest low over the specified period. If the current price exceeds the highest high, it triggers buying. If it falls below the lowest low, it triggers selling.'
        }
    },
    "VolatilityBreakout": {
        'lookback_period': {
            'description': 'The number of periods to look back for calculating the highest high and lowest low. It identifies the highest high and lowest low over the specified period.'
        },
        'volatility_factor': {
            'description': 'The factor used to multiply the highest high and lowest low. It determines the breakout levels based on the volatility. Higher factors result in wider breakout levels.'
        }
    }
}

### Insertion of the strategy info in MongoDB

In [29]:
# Create a list to store strategy data
strategy_data_list = []

# Iterate over the strategy classes
for i, (strategy_name, strategy_class) in enumerate(strategy_classes.items()):
    # Get the description of the strategy
    strategy_description = strategy_descriptions[strategy_name]

    # Get the parameters and descriptions of the strategy
    strategy_parameters_description = strategy_parameters[strategy_name]

    # Create strategy data dictionary
    strategy_data = {
        'users': "all",
        'strategy_id': strategy_name,
        'description': strategy_description,
        'parameters_description': strategy_parameters_description,
    }

    # Append strategy data to the list
    strategy_data_list.append(strategy_data)

# Perform bulk insert
db['default_strategies'].insert_many(strategy_data_list)

In [16]:
for i, (module_name, strategy_class) in enumerate(strategy_classes.items()):
    print(i, (module_name, strategy_class))

0 ('AverageDirectionalMovement', <class 'modules.AverageDirectionalMovement.AverageDirectionalMovement'>)
1 ('BollingerBands', <class 'modules.BollingerBands.BollingerBands'>)
2 ('Breakout', <class 'modules.Breakout.Breakout'>)
3 ('MACD', <class 'modules.MACD.MACD'>)
4 ('MeanReversion', <class 'modules.MeanReversion.MeanReversion'>)
5 ('MeanReversionBollinger', <class 'modules.MeanReversionBollinger.MeanReversionBollinger'>)
6 ('MomentumVolatility', <class 'modules.MomentumVolatility.MomentumVolatility'>)
7 ('MovingAverageCrossover', <class 'modules.MovingAverageCrossover.MovingAverageCrossover'>)
8 ('RelativeStrengthIndex', <class 'modules.RelativeStrengthIndex.RelativeStrengthIndex'>)
9 ('Turtle', <class 'backtesting.backtesting.Strategy'>)
10 ('VolatilityBreakout', <class 'backtesting.backtesting.Strategy'>)


# END